In [1]:
import pickle
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss, hamming_loss, accuracy_score, f1_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import word_tokenize
from PIL import Image
from tqdm import tqdm
import gc
nltk.download('punkt')
plt.rcParams['figure.figsize'] = (10, 8)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp -r "/content/drive/Shareddrives/CIS 522 Final Project/shopee-product-matching.zip" .
!unzip "/content/shopee-product-matching.zip"

In [4]:
pd.read_csv('/content/drive/Shareddrives/CIS 522 Final Project/Data/triplet_train.csv').head()

,posting_id_anchor,image_anchor,image_phash_anchor,title_anchor,label_group_positive,posting_id_positive,image_positive,image_phash_positive,title_positive,posting_id_negative,image_negative,image_phash_negative,title_negative,label_group_negative
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,train_2278313361,f83b49a86a0ee8592e3bf0204da3fbdf.jpg,ac63931c3d4b42f6,PAPER BAG VICTORIA SECRET,train_3913190043,7da59b1a67f68341b28670feeb76dfb2.jpg,fc13826cf03a9673,OTAJI Oseng Tuna Asap 500gr,2963928868
1,train_2278313361,f83b49a86a0ee8592e3bf0204da3fbdf.jpg,ac63931c3d4b42f6,PAPER BAG VICTORIA SECRET,249114794,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,train_1007590256,1a3013542c558927db0d8c1de61690a1.jpg,a8049efee5c1627a,{LAMPU TUMBLR / LAMPU NATAL / TWINKLE LIGHT / ...,2014040846
2,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,train_3423213080,8cbe4bf9706bc177fd61071ef776be8c.jpg,bfc6d01bc72c1d30,Double Tape VHB 3M ORIGINAL 12mm x 4.5mm Busa ...,train_3069587451,da80703ef767b8650acbee00f358280d.jpg,93e67c6a6159a6c1,Yazole 318 Jam Tangan Pria Original Business Q...,80347885
3,train_3423213080,8cbe4bf9706bc177fd61071ef776be8c.jpg,bfc6d01bc72c1d30,Double Tape VHB 3M ORIGINAL 12mm x 4.5mm Busa ...,2937985045,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",train_3446162576,c86c2afa855bc03a79f14bde79548ffe.jpg,dea1c13e523ea3a1,Garnier Color Naturals Express Creme 3 - Cokla...,2910074820
4,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,train_3803689425,75dbd1e9f31f2d0f21d31c08b3e0b94e.jpg,eb969469cd384ac6,Maling Ham Pork Luncheon Meat TTS 397gr,train_4248829581,5107e9669190bbf101ebdbfabb1acab5.jpg,f0aab378aa113e72,[Per Pc] Liptint Sasimi Aloe Vera 99%,1544174053


In [5]:
train_dataset = pd.read_csv('/content/drive/Shareddrives/CIS 522 Final Project/Data/triplet_train.csv')
nlp_model = Doc2Vec.load('/content/drive/Shareddrives/CIS 522 Final Project/Models/d2v.model')

In [6]:
train_dataset = train_dataset.drop_duplicates(subset=['posting_id_anchor'])

In [7]:
train_dataset, valid_dataset = train_test_split(train_dataset, test_size=0.10, random_state=1)

In [8]:
train_dataset = train_dataset.drop_duplicates(subset=['posting_id_anchor'])
valid_dataset = valid_dataset.drop_duplicates(subset=['posting_id_anchor'])

In [9]:
train_labels = train_dataset['label_group_positive']
valid_labels = valid_dataset['label_group_positive']

In [10]:
train_image_inputs = np.array([np.asarray(Image.open('train_images/{}'.format(image)).resize((224, 224))).flatten() for image in train_dataset['image_anchor']])
valid_image_inputs = np.array([np.asarray(Image.open('train_images/{}'.format(image)).resize((224, 224))).flatten() for image in valid_dataset['image_anchor']])

In [11]:
with open('/content/drive/Shareddrives/CIS 522 Final Project/ml_train_image_inputs.npy', 'wb') as f:
  np.save(f, train_image_inputs)
with open('/content/drive/Shareddrives/CIS 522 Final Project/ml_valid_image_inputs.npy', 'wb') as f:
  np.save(f, valid_image_inputs)

In [12]:
train_image_inputs = np.load('/content/drive/Shareddrives/CIS 522 Final Project/ml_train_image_inputs.npy')
valid_image_inputs = np.load('/content/drive/Shareddrives/CIS 522 Final Project/ml_valid_image_inputs.npy')

In [13]:
train_labels = train_labels
valid_labels = valid_labels

In [15]:
gc.collect()

166

In [16]:
train_text_inputs = np.array([nlp_model.infer_vector(word_tokenize(text.lower())) for text in train_dataset['title_anchor']])
valid_text_inputs = np.array([nlp_model.infer_vector(word_tokenize(text.lower())) for text in valid_dataset['title_anchor']])

In [17]:
train_inputs = np.concatenate((train_image_inputs, train_text_inputs), axis=1)
valid_inputs = np.concatenate((valid_image_inputs, valid_text_inputs), axis=1)

In [18]:
lr = KNeighborsClassifier(n_jobs=-1)
lr.fit(train_image_inputs, train_labels)

results = lr.predict(valid_image_inputs)

In [19]:
accuracy_score(valid_labels, results)

0.01

In [20]:
f1_score(valid_labels, results, average='micro')

0.01

In [21]:
f1_score(valid_labels, results, average='macro')

0.006896551724137931